<a href="https://colab.research.google.com/github/Araslt/spark/blob/main/LD02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
# SparkContext yra pagrindinė klasė, kuri valdo Spark aplikaciją ir nustato ką ji turėtų daryti.
# SparkConf yra klasė, kuri leidžia konfigūruoti Spark aplikacijos nustatymus.
# 6ios klasės importuojamos, kad būtų galima sukurti SparkContext objektą, kuris naudojamas valdyti Spark aplikaciją ir SparkConf objektą
from pyspark import SparkContext, SparkConf
# sc.stop()  # Stop the current SparkContext object
conf = SparkConf().setAppName('MyApp')
sc = SparkContext(conf=conf)

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

################## **LD02** #############################
3. Sudarykite lentele, kurioje matytusi kiek pristatyta siuntu ("siuntu skaicius")
bei aptarnauta klientu ("Sustojimo klientu skaicius") geografinese
zonose ("geografine zona") skirtingomis savaites dienomis ("sustojimo
savaites diena").



In [ ]:
# nuskaitomas  failo turinys su Spark tekstinių failų funkcija
# sc.textFile() ir jis išsaugomas kintamajame (RDD objekte) A.
A = sc.textFile("duom_cut.txt")

# funkcija gauna eilutę iš tekstinio failo kaip argumentą ir
# grąžina sąrašą (listą) iš eilučių, kurių kiekviena eilutė yra tekstas tarp }}{{.
# gaunamas maždaug toks outputas: ['a}{b}{c}{d', 'e'], jei buvo toks "{{a}{b}{c}{d}}{{e}}"
def myfmap(line):
    line = line.strip()
    line = line[2:len(line)-2]
    return line.split('}}{{')

In [ ]:
# funkcija gauna atskiras eilutes, kurios yra gaunamos iš ankstesnės funkcijos,
# kaip argumentą ir iš jų išskiria klientų skaičių, savaitės dieną,
# geografinę zoną ir siuntų skaičių. Tuomet funkcija susikuria du tuple:
# pirmasis yra unikalus raktas, susidedantis iš zonos ir savaitės dienos,
# o antrasis yra tuple su siuntų skaičiumi ir klientų skaičiumi.
# Funkcija grąžina None, jei nebuvo rasti kliento, savaitės dienos,
# zonos ar siuntos duomenys.
# Funkcijos mymap() argumentas "stopas" yra funkcijos myfmap() gražinama reikšmė.
def mymap(stopas):
    klientai = 0
    diena = 0
    zona = None
    siunta = 0

    parstrings = stopas.split('}{')
    for parstring in parstrings:
        (vardas, reiksme) = parstring.split('=')
        if(reiksme != ''):
            if(vardas == 'Sustojimo klientu skaicius'):
                klientai=int(reiksme)
            if(vardas == 'sustojimo savaites diena'):
                diena=int(reiksme)
            if(vardas == 'siuntu skaicius'):
                siunta=int(reiksme)
            if(vardas == 'geografine zona'):
                zona=reiksme

    if(klientai != 0 and diena is not None and zona is not None and siunta != 0):
        return ((zona, diena), (siunta, klientai))
    else:
        return None


In [ ]:
# flatMap(myfmap) funkcija naudoja anksčiau apibrėžtą myfmap funkciją suskaidyti
# duomenis pagal }}{{ simbolius.
# map(mymap) funkcija naudoja anksčiau apibrėžtą mymap funkciją,
# kad apdorotų kiekvieną iš gautų eilučių.
# filter(lambda x: x is not None) funkcija atmetė tuos duomenis,
# kuriuose nebuvo rasti klientų, savaitės dienos, zonos ar siuntų duomenys.
# reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])) funkcija sumažina duomenų kiekį,
# apjungdama tokius tuple, kurių raktai sutampa ir išsaugo tų tuple reikšmes,
# sumuojant jų siuntų ir klientų skaičių.
ATS = A.flatMap(myfmap)\
    .map(mymap)\
    .filter(lambda x: x is not None)\
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))


# Išvesti rezultatus
print(ATS.take(5))

[(('Z1', 3), (618, 158)), (('Z1', 5), (232, 85)), (('Z3', 5), (9, 4)), (('Z1', 1), (267, 81)), (('Z3', 1), (8, 2))]


In [ ]:
import pandas as pd
# sugeneruoti lentelės duomenis iš Spark rezultatų
table_data = []
for ((zona, diena), (siunta, klientai)) in ATS.collect():
    row = {'zona': zona, 'savaites diena': diena, 'siuntu sk.': siunta, 'Sustojimo klientu sk.': klientai}
    table_data.append(row)

# sukurti pandas lentelę su gautais duomenimis
df = pd.DataFrame(table_data)
print(df)




   zona  savaites diena  siuntu sk.  Sustojimo klientu sk.
0    Z1               3         618                    158
1    Z1               5         232                     85
2    Z3               5           9                      4
3    Z1               1         267                     81
4    Z3               1           8                      2
5    Z3               3          40                     16
6    Z1               2         607                    137
7    Z1               0          92                     18
8    Z1               4         368                     91
9    Z3               0           4                      3
10   Z3               2          17                      8
11   Z3               4          10                      6


In [ ]:
# substitute
def mymap(stopas):
    mapping = {}
    parstrings = stopas.split('}{')
    for parstring in parstrings:
        (vardas, reiksme) = parstring.split('=')
        if reiksme:
            mapping[vardas] = reiksme
    if all(k in mapping for k in ['Sustojimo klientu skaicius', 'sustojimo savaites diena', 'siuntu skaicius', 'geografine zona']):
        return ((mapping['geografine zona'], int(mapping['sustojimo savaites diena'])), (int(mapping['siuntu skaicius']), int(mapping['Sustojimo klientu skaicius'])))
    else:
        return None
# print(mymap('geografine zona=1}{sustojimo savaites diena=2}{siuntu skaicius=3}{Sustojimo klientu skaicius=4'))


(('1', 2), (3, 4))


#
#
#
################## **END LD02** #############################

In [ ]:
# spark is an existing SparkSession
df = spark.read.json("people.json")
# Displays the content of the DataFrame to stdout
df.show()
# +----+-------+
# | age|   name|
# +----+-------+
# |null|Michael|
# |  30|   Andy|
# |  19| Justin|
# +----+-------+

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [ ]:
from pyspark.sql.types import *
sc = spark.sparkContext
lines = sc.textFile("people.txt")
parts = lines.map(lambda l: l.split(","))
people = parts.map(lambda p: Row(name=p[0],age=int(p[1])))
peopledf = spark.createDataFrame(people)

In [ ]:
people = parts.map(lambda p: Row(name=p[0], age=int(p[1].strip())))
# schemaString = "name age"
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)
spark.createDataFrame(people, schema).show()

In [ ]:
A = sc.textFile("duom_cut.txt")
def myfmap(line):
  line = line.strip()
  line = line[2:len(line)-2]
  return line.split('}}{{')

def mymap(stopas):
  #Sustojimo klientu skaicius
  klientai = 0
  #sustojimo savaites diena
  diena = None
  parstrings = stopas.split('}{')
  for parstring in parstrings:
    (vardas, reiksme) = parstring.split('=')
    if(reiksme != ''):
      if(vardas == 'Sustojimo klientu skaicius'):
        klientai=int(reiksme)
      if(vardas == 'sustojimo savaites diena'):
        diena=int(reiksme)
  #if(klientai != None and diena != None):
  return (diena, klientai)
ATS = A.flatMap(myfmap)\
  .map(mymap)\
  .filter(lambda kv : kv[0]!=None)

#diena != None
#print(sustojimai.take(10))
print(ATS.take(10))
dfl3 = spark.createDataFrame(ATS.map(lambda pora:Row(diena=pora[0],klientai=pora[1])))
dfl3.show()

[(2, 1), (2, 1), (2, 1), (2, 1), (2, 1), (2, 1), (2, 1), (2, 0), (3, 1), (3, 1)]
+-----+--------+
|diena|klientai|
+-----+--------+
|    2|       1|
|    2|       1|
|    2|       1|
|    2|       1|
|    2|       1|
|    2|       1|
|    2|       1|
|    2|       0|
|    3|       1|
|    3|       1|
|    3|       1|
|    3|       1|
|    3|       2|
|    3|       7|
|    3|       1|
|    3|       1|
|    3|       9|
|    4|       2|
|    4|       1|
|    4|       0|
+-----+--------+
only showing top 20 rows



In [ ]:
#from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Laboras3').getOrCreate()


from pyspark.sql.functions import udf, log
from pyspark.sql.types import *

#nuo spark 3.0
#spark.conf.set("spark.sql.legacy.setCommandRejectsSparkCoreConfs","false")

#spark.conf.set("spark.dynamicAllocation.enabled", "true")
#spark.conf.set("spark.executor.instances", 4)
#spark.conf.set("spark.executor.cores", 4)

text_file = spark.sparkContext.textFile("duom_cut.txt")
#text_file = sc.textFile("hdfs:///user/stud0/labt/duom_cut.txt")

def parsinam(line):
	#line=line[2:len(line)-2]
	return line[2:len(line)-2].split('}}{{')

def parsinam2(line):
	objs = line.split('}{')
	k1=None
	k3=None
	for at in objs:
		temp = at.split('=')
		if(len(temp)<2):
			break
		key,val=at.split('=')
		if(key == 'marsrutas'):
			k1=val
		if(key == 'sustojimo data'):
			k3=val
	if(k1!=None and k3!=None):
		return (k1+"_"+k3,(k3,len(str(k3))))
	#else:
	#	return (0,(0,0))

fmap = text_file.flatMap(parsinam)
#fmap = text_file.flatMap(lambda line:line[2:len(line)-2].split('}}{{'))
mmap = fmap.map(parsinam2)

##Jusu darbas cia:

##Kodas, kito failo nuskaitymas ... duomenu agregavimas
routes = spark.read.option("header",True).csv("RouteSummary.txt")
routes.printSchema()
routes = routes.drop("M", "BendrasAtstumas","BendrasLaikas")

def makeID(str1, str2):
    return str1+"_"+str2

makeID_UDF = udf(lambda z1,z2: makeID(z1,z2),StringType())

routes2 = routes.withColumn('ID', makeID_UDF("marsrutas", "sustojimo data")).drop("marsrutas", "sustojimo data")
routes2.printSchema()

routes2.write.csv("lentele3")

#listas = routes2.rdd.collect()
#textfile = open("out.txt", "w")
#for element in listas:
#	textfile.write(element + "\n")
#textfile.close()






##training data formato: ("prognozuojama reiskme", "parametras")
training = mmap.toDF()

#regression ...

from pyspark.ml.regression import LinearRegression

# Load training data
#training = spark.read.format("libsvm")\
#    .load("sample_linear_regression_data.txt")

lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(training)

# Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)




#dfFromRDD1.show()
#training = training.withColumnRenamed('marsrutas', 'parametrai')


training.printSchema()

pandasDF = training.toPandas()
pandasDF.head()

labels = pandasDF['_1'].to_list()

values = pandasDF['_2'].to_list()
print(labels)
print(values)

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from mpl_toolkits.mplot3d import Axes3D

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(20,5))
axes[2].scatter(labels,labels, s = 10)

#training.plot.scatter(x='label',
#                      y='label',
#                      c='ats',colormap='viridis',ax=axes[0])
plt.show()

In [ ]:
#########################################################

In [ ]:
# A bus RDD objektas
A = sc.textFile("ket.txt").map(lambda line: line.strip().split())
print(A.collect())
B = A.filter(lambda x: x[1] == 'greitis')
print("B", B.collect())
B = B.map(lambda x: (x[2],1))
print("B", B.collect())
B = B.reduceByKey(lambda a, b: a+b)
print("B", B.collect())
C = A.map(lambda x: (x[2], int(x[1] == 'greitis')))
print("C", C.collect())
C = C.filter(lambda x: x[1] != 0 )
C = C.reduceByKey(lambda a, b: a+b)
print("C", C.collect())

[['1', 'greitis', '1111'], ['2', 'kita', '1111'], ['3', 'kita', '2222'], ['4', 'greitis', '2222'], ['5', 'kita', '3333'], ['6', 'greitis', '2222'], ['7', 'greitis', '2222']]
B [['1', 'greitis', '1111'], ['4', 'greitis', '2222'], ['6', 'greitis', '2222'], ['7', 'greitis', '2222']]
B [('1111', 1), ('2222', 1), ('2222', 1), ('2222', 1)]
B [('2222', 3), ('1111', 1)]
C [('1111', 1), ('1111', 0), ('2222', 0), ('2222', 1), ('3333', 0), ('2222', 1), ('2222', 1)]
C [('2222', 3), ('1111', 1)]


Nagrinekime duomenis, aprašancius KET pažeidimus. Tarkime, kad Spark terpeje egzistuoja duomenys
A RDD pavidalu, duomenys aprašomi šitais atributais:
(a) pažeidimo numeris – sveikas skaicius,
(b) pažeidimo rušis – žodis (string),
(c) pažeidejo kodas (ID) – sveikas skaicius.
Užrašykite Spark kodo fragmenta, kuris suskaiciuotu kiek skirtingi pažeidejai turi rušies "greitis"
pažeidimu.

In [ ]:
A = sc.textFile("ket.txt").map(lambda line: line.strip().split())
B = A.filter(lambda list: (list[1] == 'greitis')).map(lambda list: (list[2],1))\
.reduceByKey(lambda a, b: a+b)
print(B.collect())
B = A.map(lambda list: (list[2], int(list[1] == 'greitis')))\
.reduceByKey(lambda a, b: a+b)
print(B.collect())
# print(typr(take(3)))

[('2222', 3), ('1111', 1)]
[('2222', 3), ('1111', 1), ('3333', 0)]


Nagrinekime duomenis, aprašancius poliklinikos pacientu duomenis – pacientu apsilankymus del
ligos. Tarkime, kad Spark terpeje egzistuoja duomenys A RDD pavidalu, duomenys aprašomi šitais
atributais:
(a) Apsilankymo metai – sveikas skaicius,
(b) ligos pavadinimas – žodis (string),
(c) paciento kodas (ID) – sveikas skaicius.
Užrašykite Spark kodo fragmenta, kuris parodytu ar 2019 metais buvo daugiau pacientu apsilankusiu
del ligos "tuberkulioze" nei 2018 metais (atsakymas gali buti "taip"/"ne" pavidalu). Pastaba: jeigu
tas pats pacientas apsilanke kelis kartus, laikoma kad tai vienas pacientas, apsilankes del ligos tais
metais.

In [ ]:
A = sc.textFile('ligos.txt').map(lambda x: x.strip().split())
B = A.filter(lambda metai: metai[0] == '2018')
print('B', B.collect())
B = B.map(lambda x: (x[2], 1))
print('B', B.collect())
# nera sumavimo a+b, tai bus tik uniklaios reiksmes
B = B.reduceByKey(lambda a, b: a)
print('B', B.collect())
print(B.count())
# visi raktai paversti i 1, kad veliau suskaiciuoti suma
# B = B.map(lambda x: 'a')
B = B.map(lambda x: 1)
print('B', B.collect())
B = B.reduce(lambda a, b: a+b)
print('B', B)
C = A.filter(lambda metai: metai[0] == '2019') \
.map(lambda x: (x[2], 1)) \
.reduceByKey(lambda a, b: a) \
.map(lambda id: 1) \
.reduce(lambda a, b: a+b)
# paskutini reduce galima pakeisti .count()
print("C", C)
print('tap' if C>B else "ne")

D = A.map(lambda l: (l[0], l[2], int(l[1] == 'tuberkulioze')))
print(D.collect())

B [['2018', 'liga2', '2222'], ['2018', 'tuberkulioze', '1111']]
B [('2222', 1), ('1111', 1)]
B [('2222', 1), ('1111', 1)]
2
B [1, 1]
B 2
C 3
tap
[('2020', '1111', 1), ('2020', '2222', 1), ('2020', '1111', 1), ('2019', '1111', 1), ('2019', '2222', 1), ('2019', '1111', 1), ('2019', '3333', 0), ('2019', '1111', 0), ('2018', '2222', 0), ('2018', '1111', 1)]


In [ ]:
A = sc.textFile('ligos.txt').map(lambda line: line.strip().split())
B = A.filter(lambda metai: (metai[0] == '2018'))\
  .map(lambda l: (l[2],1))\
  .reduceByKey(lambda a, b:a+b)
print(B.collect())
print(B.count())
co = A.filter(lambda metai: (metai[0] == '2018'))\
  .map(lambda ld: (ld[2],1)).reduceByKey(lambda a, b: a)\
  .map(lambda ld: 1).reduce(lambda x, y: x+y)
print(co)

co19 = A.filter(lambda metai: (metai[0] == '2019'))\
  .map(lambda ld: (ld[2],1)).reduceByKey(lambda a, b: a).count()
print('taip' if co19 > co else 'ne')

B=A.map(lambda l : ((l[0],l[2]),int(l[1]=='tuberkulioze')))\
  .reduceByKey(lambda a,b: max(a,b)).map(lambda l:(l[0][0],l[0][1],l[1]))\
  .reduceByKey(lambda a,b:a+b)
print(B)

[('2222', 1), ('1111', 1)]
2
2
taip
PythonRDD[496] at RDD at PythonRDD.scala:53


3. Nagrinekime duomenis apie vaistines, saugojancias vaistus, duomenys aprašomi šitais atributais:
(a) vaistines ID – sveikas skaicius,
(b) vaistu sarašas, vaistai aprašomi šitais atributais:
i. pagrindine veiklioji medžiaga – žodis (string),
ii. kiekis – realusis skaicius,
iii. galiojimo termino data – datos pavidalu, paprastumo delei galite laikyti, kad nurodomi tik
metai.
Išspreskite šiuos uždavinius:
(a) Kiekvienai vaistinei išveskite joje esanciu veikliuju medžiagu saraša.

In [ ]:
import json

def convertVaistine(vaistine):
  id=0
  try:
    id=int(vaistine[0])
  except ValueError:
    id=0
  sar = json.loads(vaistine[1])
  S = set()
  for el in sar:
    S.add(el[0])
  return [id,list(S)]

A = sc.textFile('vaistines.txt').map(lambda line: line.strip().split())\
  .map(convertVaistine)
sample = A.collect()
print(sample)

[[1, ['B', 'C', 'A']], [2, ['B', 'C', 'A']], [3, ['B', 'C', 'A']], [5, ['B', 'C', 'A']], [4, ['B', 'C', 'A', 'D']]]


In [ ]:
import json

def fmap(vaistine):
  id=0
  try:
    id=int(vaistine[0])
  except ValueError:
    id=0
  sar = json.loads(vaistine[1])
  ats = []
  for el in sar:
    ats.append(((id,el[0]), 1))
  return ats

A = sc.textFile('vaistines.txt').map(lambda line: line.strip().split())\
  .flatMap(fmap).reduceByKey(lambda a,b:a)\
  .map(lambda l:(l[0][0],l[0][1])).reduceByKey(lambda a,b: a+b)\
  .groupByKey().map(lambda l:(l[0],list(l[1])))

# print([(id, list(s)) for id,s in A.collect()])
print(A.collect())

[(2, ['CAB']), (4, ['CABD']), (1, ['ABC']), (3, ['ABC']), (5, ['ABC'])]


(b) Raskite vaistus, kuriu galiojimo terminas pasibaige.


In [ ]:
A = sc.textFile('vaistines.txt').map(lambda line: line.strip().split())\
  .map(convertVaistine)

def fmap(vaistine):
  id=0
  try:
    id=int(vaistine[0])
  except ValueError:
    id=0
  sar = json.loads(vaistine[1])
  return sar

A = sc.textFile('vaistines.txt').map(lambda line: line.strip().split())\
  .flatMap(fmap)

print(A.take(3))

[['A', 50, 2022], ['A', 15, 2020], ['B', 50, 2025]]


dfsd

In [ ]:
A = sc.textFile('vaistines.txt').map(lambda line: line.strip().split())\
  .map(convertVaistine)

# def fmap(vaistine):
#   id=0
#   try:
#     id=int(vaistine[0])
#   except ValueError:
#     id=0
#   sar = json.loads(vaistine[1])
#   return sar

A = sc.textFile('vaistines.txt').map(lambda line: line.strip().split())\
  .flatMap(lambda vaistine: json.loads(vaistine[1])).filter(lambda vaistas:vaistas[2]<2023)

print(A.take(3))

[['A', 50, 2022], ['A', 15, 2020], ['C', 150, 2021]]


(c) Raskite vaistines, kuriose yra vaistu su pasibaigusiu galiojimo terminu.

In [ ]:
A = sc.textFile('vaistines.txt').map(lambda line: line.strip().split())\
  .map(convertVaistine)

def fmap(vaistine):
  id=0
  try:
    id=int(vaistine[0])
  except ValueError:
    id=0
  sar = json.loads(vaistine[1])
  ats = []
  for el in sar:
    ats.append((id,int(el[2]<2023)))
  return ats

A = sc.textFile('vaistines.txt').map(lambda line: line.strip().split())\
  .flatMap(fmap).reduceByKey(lambda a,b:a+b).filter(lambda vaistine:vaistine[1]>0)

# print([(id, list(s)) for id,s in A.collect()])
print(A.collect())

[(2, 4), (4, 5), (1, 4), (3, 2)]


(d) Kiekvienai veikliajai medžiagai paskaiciuokite vaistu (iš visu vaistiniu) su pasibaigusiais galiojimo
terminais dali.

In [ ]:
A = sc.textFile('vaistines.txt').map(lambda line: line.strip().split())\
  .map(convertVaistine)
A = sc.textFile('vaistines.txt').map(lambda line: line.strip().split())\
  .flatMap(lambda vaistine: json.loads(vaistine[1]))\
  .map(lambda vaistas: (vaistas[0],(vaistas[1], vaistas[1] if vaistas[2]<2023 else 0)))\
  .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))\
  .map(lambda vaistas: (vaistas[0], vaistas[1][1]/vaistas[1][0]))

print(A.collect())

[('C', 0.849624060150376), ('A', 0.7678571428571429), ('B', 0.07407407407407407), ('D', 1.0)]


(e) Raskite 10% vaistiniu su didžiausiais vaistu (iš visu vaistiniu) su pasibaigusiais galiojimo terminais
skaiciu ir bendru vaistu skaiciu santykiais.

In [ ]:
def fmap(vaistine):
  id=0
  try:
    id=int(vaistine[0])
  except ValueError:
    id=0
  sar = json.loads(vaistine[1])
  ats = []
  for el in sar:
    ats.append((id,int(el[1],el[1] if int(el([2]<2023) else 0)))
  return ats

A = sc.textFile('vaistines.txt').map(lambda line: line.strip().split())\
  .flatMap(fmap).

LD02
Sudarykite lentele, kurioje matytusi kiek pristatyta siuntu ("siuntu skaicius") bei aptarnauta klientu ("Sustojimo klientu skaicius") geografinese zonose ("geografine zona") skirtingomis savaites dienomis ("sustojimo savaites diena").

In [ ]:
# Užduoties numeris
n = 10
result = (n % 4) + 1
print(result)

3


In [ ]:
import pandas as pd

# nuskaitomas  failo turinys su Spark tekstinių failų funkcija
# sc.textFile() ir jis išsaugomas kintamajame (RDD objekte) A.
A = sc.textFile("text2.txt")

# funkcija gauna eilutę iš tekstinio failo kaip argumentą ir
# grąžina sąrašą iš eilučių, kurių kiekviena eilutė yra tekstas tarp {{ ir }}.
# Ši funkcija yra naudojama suskaidant duomenis pagal {{ ir }} simbolius.
def myfmap(line):
    line = line.strip()
    line = line[2:len(line)-2]
    return line.split('}}{{')

# funkcija gauna atskiras eilutes, kurios yra gaunamos iš ankstesnės funkcijos,
# kaip argumentą ir iš jų išskiria klientų skaičių, savaitės dieną,
# geografinę zoną ir siuntų skaičių. Tuomet funkcija susikuria du tuple:
# pirmasis yra unikalus raktas, susidedantis iš zonos ir savaitės dienos,
# o antrasis yra tuple su siuntų skaičiumi ir klientų skaičiumi.
# Funkcija grąžina None, jei nebuvo rasti kliento, savaitės dienos,
# zonos ar siuntos duomenys.
# Funkcijos mymap() argumentas "stopas" yra funkcijos myfmap() gražinama reikšmė.
def mymap(stopas):
    klientai = 0
    diena = 0
    zona = None
    siunta = 0

    parstrings = stopas.split('}{')
    for parstring in parstrings:
        (vardas, reiksme) = parstring.split('=')
        if(reiksme != ''):
            if(vardas == 'Sustojimo klientu skaicius'):
                klientai=int(reiksme)
            if(vardas == 'sustojimo savaites diena'):
                diena=int(reiksme)
            if(vardas == 'siuntu skaicius'):
                siunta=int(reiksme)
            if(vardas == 'geografine zona'):
                zona=reiksme

    if(klientai != 0 and diena is not None and zona is not None and siunta != 0):
        return ((zona, diena), (siunta, klientai))
    else:
        return None

# flatMap(myfmap) funkcija naudoja anksčiau apibrėžtą myfmap funkciją suskaidyti
# duomenis pagal {{ ir }} simbolius.
# map(mymap) funkcija naudoja anksčiau apibrėžtą mymap funkciją,
# kad apdorotų kiekvieną iš gautų eilučių.
# filter(lambda x: x is not None) funkcija atmetė tuos duomenis,
# kuriuose nebuvo rasti klientų, savaitės dienos, zonos ar siuntų duomenys.
# reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])) funkcija sumažina duomenų kiekį,
# apjungdama tokius tuple, kurių raktai sutampa ir išsaugo tų tuple reikšmes,
# sumuojant jų siuntų ir klientų skaičių.
ATS = A.flatMap(myfmap)\
    .map(mymap)\
    .filter(lambda x: x is not None)\
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))


# Išvesti rezultatus
print(ATS.collect())


# sugeneruoti lentelės duomenis iš Spark rezultatų
table_data = []
for ((zona, diena), (siunta, klientai)) in ATS.collect():
    row = {'zona': zona, 'savaites diena': diena, 'siuntu sk.': siunta, 'Sustojimo klientu sk.': klientai}
    table_data.append(row)

# sukurti pandas lentelę su gautais duomenimis
df = pd.DataFrame(table_data)
print(df)




[(('Z1', 3), 165), (('Z1', 2), 13), (('Z1', 0), 12), (('Z1', 4), 153)]
  zona  savaites diena  siuntu sk.
0   Z1               3         165
1   Z1               2          13
2   Z1               0          12
3   Z1               4         153


In [ ]:
B = open("text.txt", "r")
count = 0
for line in B:
  line = line.strip()
  line = line[2:len(line) - 2]
  count += 1
  susstring = line.split('}}{{')
print(count)

65


In [ ]:
A = sc.textFile("text.txt")
def myfmap(line):
  line = line.strip()
  line = line[2:len(line)-2]
  return line.split('}}{{')
def mymap(stopas):
  #Sustojimo klientu skaicius
  klientai = 0
  #sustojimo savaites diena
  diena = None
  parstrings = stopas.split('}{')
  for parstring in parstrings:
    (vardas, reiksme) = parstring.split('=')
    if(reiksme != ''):
      if(vardas == 'Sustojimo klientu skaicius'):
        klientai=int(reiksme)
      if(vardas == 'sustojimo savaites diena'):
        diena=int(reiksme)
  #if(klientai != None and diena != None):
  return (diena, klientai)
ATS = A.flatMap(myfmap)\
  .map(mymap)\
  .filter(lambda kv : kv[0]!=None)\
  .reduceByKey(lambda a,b:a+b)
#diena != None
#print(sustojimai.take(10))
print(ATS.take(10))
print(ATS.collect())

[(2, 163), (4, 108), (3, 201), (5, 109), (1, 90)]
[(2, 163), (4, 108), (3, 201), (5, 109), (1, 90)]


In [ ]:
x = sc.parallelize(["b", "a", "c"])
y = x.map(lambda z: (z, 1))

def ma(z):
  return [z, 1]

print(x.collect())
print(y.collect())
# print(y2.collect())

['b', 'a', 'c']
[('b', 1), ('a', 1), ('c', 1)]


In [ ]:
# A = sc.textFile('pvz.txt')
A = sc.parallelize(["b", "a", "c"])
print(type(A.collect()))
# def fum(w):
#   if w!='b':
#     return [w]
#   else:
#     return []
y = A.flatMap(lambda z: [z] if z != 'b' else [])
y2 = A.filter(lambda w: w != 'b')
y3 = A.map(lambda v: v != 'b')
print(y.collect())
print(y2.collect())
print(y3.collect())
# y = A.flatMap(fum)
# print(A.collect())
# print(y.collect())
# y2 = A.filter(lambda w : w!='b')
# print(y2.collect())


<class 'list'>
['a', 'c']
['a', 'c']
[False, True, True]


In [ ]:
# x = sc.parallelize([1,2,3])
x = sc.textFile("int.txt")\
.map(lambda l: int(l))
y = x.filter(lambda o: o%2 == 1) #keep odd values
print(x.collect())
print(y.collect())

[1, 2, 3, 56, 12, 5644, 654, 52165, 2, 65, 36321, 5]
[1, 3, 52165, 65, 36321, 5]


In [ ]:
x = sc.parallelize([1,2,3])
y = x.map(lambda x: [x, x*100, 42])
fy = x.flatMap(lambda x: [x, x*100, 42])
print(x.collect())
print(y.collect())
print(fy.collect())

[1, 2, 3]
[[1, 100, 42], [2, 200, 42], [3, 300, 42]]
[1, 100, 42, 2, 200, 42, 3, 300, 42]


In [ ]:
A = sc.textFile("text2.txt")
words = A.flatMap(lambda line: line.strip().split())
print(words.take(10))
pomap = words.map(lambda w: (w, 1))
print("pomap:", pomap.take(2))

def red(a, b):
  return a+b
pored = pomap.reduceByKey(red)
print(pored.collect())

B = A.flatMap(lambda line: line.strip().split())\
.map(lambda w: (w, 1))\
.reduceByKey(red)
print(B.collect())
# print([(k, list(v)) for (k, v) in B.collect])

['Within', 'two', 'minutes,', 'or', 'even', 'less,', 'he', 'had', 'forgotten', 'all']
pomap: [('Within', 1), ('two', 1)]
[('Within', 1), ('two', 1), ('minutes,', 1), ('even', 2), ('less,', 1), ('he', 6), ('his', 11), ('troubles.', 1), ('Not', 1), ('troubles', 1), ('heavy', 1), ('than', 2), ('are', 2), ('but', 1), ('new', 5), ('interest', 2), ('them', 2), ('down', 2), ('drove', 1), ('out', 1), ('of', 11), ('mind', 1), ('as', 4), ("men's", 1), ('in', 6), ('excitement', 1), ('enterprises.', 1), ('was', 11), ('valued', 1), ('whistling,', 1), ('just', 1), ('acquired', 1), ('suffering', 1), ('It', 2), ('consisted', 1), ('bird-like', 1), ('produced', 1), ('touching', 1), ('mouth', 2), ('at', 3), ('short', 1), ('intervals', 1), ('midst', 1), ('music—the', 1), ('probably', 1), ('remembers', 1), ('do', 1), ('it,', 2), ('ever', 1), ('Diligence', 1), ('gave', 1), ('knack', 1), ('strode', 1), ('full', 2), ('harmony', 1), ('felt', 1), ('an', 2), ('feels', 1), ('far', 1), ('strong,', 1), ('deep,', 1)